<a href="https://colab.research.google.com/github/igormatheusdiniz/ri_lab_07/blob/master/ri_lab7_igor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [250]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import heapq as hp
import math
import bisect

import nltk
nltk.download('rslp')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import heapq as hp

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Leitura e tratamento dos dados do .csv e Json

In [0]:
url = 'https://raw.githubusercontent.com/Benardi/ri_lab_01/master/output/results.csv'

In [0]:
dataframe = pd.read_csv(url)
dataframe['treatedText'] = dataframe.text.apply(lambda x: re.sub('[^0-9A-ZÀ-Úa-zà-ú\']', ' ', x.lower()))

In [0]:
regex = RegexpTokenizer(r'[A-zÀ-ú-]+')
stopWords = set(stopwords.words('portuguese'))

In [0]:
urlJson = "https://raw.githubusercontent.com/igormatheusdiniz/ri_lab_07/master/results_final.json"
json = pd.read_json(urlJson)
feedback = {json['query'][i]:json['docs'][i] for i in range(10)}

# Construção do Índice Invertido

In [0]:
def ocurrences(list, token):
  print(list)
  print(token)
  count = 0
  for word in list:
    if word==token:
      count +=1
  return count
      

def invertedIndex(documents):
  I = {}
  index = 0
  
  ##Para cada documento na minha relação de documentos irei realizar o processo de tokenização
  for document in documents.treatedText:
    index += 1
    T = [token for token in regex.tokenize(document)
        if((len(token)>=3) and token not in stopWords and token[0] != "[" and token[-1]!="]")]
    
    ##Por motivos didáticos, resolvi ordenar minha lista de tokens
    #T.sort()
    
    ## Pra cada token criado se ele ainda não existe na minha estrutura de armazenamento como uma chave, eu crio e em seguida adiciono
    ##o indice do documento como valor para esta chave
    ## Caso já exista, apenas adiciona o índice do documento para esta chave
    for token in T:
      if token not in I.keys():
        I[token]=[]
       
      pair = []
      ocurrence = T.count(token)
     
      pair.append(index)
      pair.append(ocurrence)
      if(pair not in I[token]):
        I[token].append(pair)
  
  ## Calculo do IDF
  for key in I:
    idf = math.log((documents.treatedText.count()+1)/ len(I[key]))
    I[key].append(idf)
        
  ## Por motivos estéticos resolvi comentar esta linha para não ficar uma lista de print muito grande no gitHub, dificultando a visualização do documento como um todo      
  ##print(I)
  return I

## Exportando para .csv
invertedIndex = invertedIndex(dataframe)
data = {'token': list(invertedIndex.keys()), 'ocurrences': list(invertedIndex.values())}
df = pd.DataFrame(data)
df.to_csv('index.csv')

# Algoritimo necessários

In [0]:
def binarioModelDocument(document):
  vetores = []
  #for document in dataframe.treatedText:
  ## Tokenização do documento passado
  dicionario = {}
  T = [token for token in regex.tokenize(document)
      if((len(token)>=3) and token not in stopWords and token[0] != "[" and token[-1]!="]")]
    
    ##Por motivos didáticos, resolvi ordenar minha lista de tokens
  #T.sort()
  
  for indice in invertedIndex:
    dicionario[indice] = 0
    
    if indice in T :
      dicionario[indice] = 1

  return dicionario

## Metodo para calculo do Score
def calculaScore(vetorQuery, vetorDocument):
  ##Criação de dois vetores com relação ao vocabulario, query vs. vocabulario, document vs. vocabulario
  vetor_num_query = np.array(list(vetorQuery.values()))
  vetor_num_documents = np.array(list(vetorDocument.values()))
  ##Multiplicação dos vetores
  result = vetor_num_query * vetor_num_documents
  result = sum(result)
  return result


def binarioModel(consulta, document): 
  dicionario = {}
  scores = []
  vetor = binarioModelDocument(document)
  
  for indice in invertedIndex:
    dicionario[indice] = 0
    
    if indice in consulta.split() :
      dicionario[indice] = 1
  
  ## Calculo do score por modelo Binario, chamando o método auxiliar criado anteriormente
  score = calculaScore(dicionario, vetor)       
  return (score)

#binary_model = binarioModel("juíza federal",dataframe.treatedText[0])
#print(binary_model)

In [0]:
def termFrequency(terms, document):
  hashtable = {}
  query =[]
  document_vocabulary = []
  
  ## Tokenização do documento
  document_vocabulary = [token for token in regex.tokenize(document)
      if((len(token)>=3) and token not in stopWords and token[0] != "[" and token[-1]!="]")]
  
  ## Construção do vetor de termos em comum da query e do vocabulario
  for term in terms.split():
    if term in invertedIndex.keys():
      query.append(term)
    
  ## Contagem da frequência de acordo o palavra no vocabulario e na query de consulta  
  frequency = 0
  for word in document_vocabulary:
    if word in query:
      frequency += 1
    
  return (frequency)

#resultado = termFrequency("juíza federal", dataframe.treatedText[158])
#print(resultado)

In [0]:
def tf_idf(terms, document):
  hashtable = {}
  invertedLists ={}
  R = []
  
  document_vocabulary = [token for token in regex.tokenize(document)
                         if((len(token)>=3) and token not in stopWords and token[0] != "[" and token[-1]!="]")]
  
  for term in terms.split():
    if term in invertedIndex.keys():
      invertedLists[term] = []
      invertedLists[term] = invertedIndex[term]
  
  ## Semelhante ao metodo
  frequency = 0
  for word in document_vocabulary:
    if word in invertedLists:
      frequency += 1*invertedLists[word][-1]
  
  return frequency

#resultado = tf_idf("juíza federal", dataframe.treatedText[14])
#print(resultado)

In [0]:
N = dataframe.text.count()
def bm25(consulta, document, k):
  document_vocabulary = [token for token in regex.tokenize(document)
                         if((len(token)>=3) and token not in stopWords and token[0] != "[" and token[-1]!="]")]
  query = consulta.split()
  intersec =  list(set(document_vocabulary) & set(query))
  
  score = 0
  for word in intersec:
    cwq = query.count(word)
    cwd = document_vocabulary.count(word)
    dfw = len(invertedIndex[word][:-1])
    y = ((k+1)*cwd)/(cwd+k)
    score += cwq*y*math.log((N+1)/(dfw))
  
  return score
    
#bm25_result = bm25('forças armadas', dataframe.treatedText[149], 5)
#print(bm25_result)
  

# 1. Escolha um documento dentre aqueles da base do aluno Bernardi e crie uma consulta que você acha que tem boas chances de recuperar este documento. Em seguida, avalie os resultados de tal consulta usando a métrica de avaliação Reciprocal Rank

Escolhemos o documento 16, o mesmo fala sobre a permissão por parte do Papa para abertura dos arquivos secretos sobre o holocausto.

In [260]:
ndoc = 16
document = dataframe.loc[ndoc]
consulta = 'igreja nazista'
document.title

'Por que o papa Francisco ordenou a abertura dos arquivos secretos do Vaticano sobre o Holocausto judeu?'

In [261]:
row = dataframe.loc[dataframe.url == document.url]
row

,title,subtitle,author,date,section,text,url,treatedText
16,Por que o papa Francisco ordenou a abertura do...,Se Francisco realmente conseguir lançar luz se...,Juan Arias,04/03/2019 21:07:23,Opinião,No momento em que o papado de se vê envolvido...,https://brasil.elpais.com/brasil/2019/03/05/op...,no momento em que o papado de se vê envolvido...


In [0]:
def createTable(consulta, dataframe):
  table1 = pd.DataFrame()

  rank_binario = []
  rank_termFrequency = []
  rank_tf_idf = []
  rank_bm25 = []

  document_number = 0
  for document in dataframe.treatedText:
  
    score_binario = binarioModel(consulta, document)
    score_termFrequency = termFrequency(consulta, document)
    score_tf_idf = tf_idf(consulta, document)
    score_bm25 = bm25(consulta, document,10)
  
    rank_binario.append((document_number, score_binario))
    rank_termFrequency.append((document_number, score_termFrequency))
    rank_tf_idf.append((document_number, score_tf_idf))
    rank_bm25.append((document_number, score_bm25))
  
    document_number+=1

  rank_binario.sort(key=lambda x: x[1], reverse = True)
  rank_termFrequency.sort(key=lambda x: x[1], reverse = True)
  rank_tf_idf.sort(key=lambda x: x[1], reverse = True)
  rank_bm25.sort(key=lambda x: x[1], reverse = True)

  rank_binario = rank_binario[0:10]
  rank_termFrequency = rank_termFrequency[0:10]
  rank_tf_idf = rank_tf_idf[0:10]
  rank_bm25 = rank_bm25[0:10]

  table1["Binario"] = rank_binario
  table1["TF"] = rank_termFrequency
  table1["Tf-Idf"] = rank_tf_idf
  table1["BM25"] = rank_bm25

  return table1

In [263]:
table = createTable(consulta, dataframe)
table

,Binario,TF,Tf-Idf,BM25
0,"(7, 1)","(16, 6)","(16, 15.786534959796493)","(180, 12.25648871776738)"
1,"(16, 1)","(163, 6)","(163, 15.786534959796493)","(16, 10.853242784860088)"
2,"(17, 1)","(68, 4)","(180, 14.484941211906904)","(163, 10.853242784860088)"
3,"(28, 1)","(78, 4)","(68, 10.524356639864328)","(110, 8.851908518387553)"
4,"(68, 1)","(96, 3)","(78, 10.524356639864328)","(68, 8.269137359893401)"
5,"(78, 1)","(176, 3)","(110, 9.656627474604603)","(78, 8.269137359893401)"
6,"(92, 1)","(180, 3)","(96, 7.893267479898245)","(96, 6.678918636836977)"
7,"(96, 1)","(196, 3)","(176, 7.893267479898245)","(176, 6.678918636836977)"
8,"(100, 1)","(110, 2)","(196, 7.893267479898245)","(196, 6.678918636836977)"
9,"(110, 1)","(7, 1)","(7, 2.631089159966082)","(7, 2.631089159966082)"


In [264]:
def reciprocalRank(entry, docId):
  n = 1.0;
  for doc,r in entry:
    if doc == docId:
      return  [1 / n]
    else:
      n += 1
  return 1/n

rank = pd.DataFrame()
rank['Binary'] = reciprocalRank(table['Binario'], ndoc)
rank['TF'] = reciprocalRank(table['TF'], ndoc)
rank['TF-IDF'] = reciprocalRank(table['Tf-Idf'], ndoc)
rank['BM25'] = reciprocalRank(table['BM25'], ndoc)

rank

,Binary,TF,TF-IDF,BM25
0,0.5,1.0,1.0,0.5



Pra as abordagens TF e TF-IDF, o valor do reciprocal rank sempre foi 1, pois o documento escolhido, cujo índice é 16, foi o primeiro colocado nos resultados da consulta.

# 2. A partir do gabarito fornecido em OBS1, calcule o MAP para cada algoritmo abaixo e aponte qual obteve o melhor resultado. Para os cálculos do MAP, considere que um documento é relevante para uma dada consulta se este documento estiver entre os documentos do gabarito para essa consulta, senão ele deve ser considerado irrelevante.

In [0]:
def create_models(query,k):
  binario = []
  tf = []
  tfidf = []
  BM25 = []
  for i in range(len(dataframe)):
    doc = dataframe.treatedText[i]
    bisect.insort(binario, (binarioModel(query, doc), i))
    bisect.insort(tf, (termFrequency(query,doc), i))
    bisect.insort(tfidf, (tf_idf(query,doc), i))
    bisect.insort(BM25, (bm25(query,doc,20), i))
  
  binario.reverse()
  tf.reverse()
  tfidf.reverse()
  BM25.reverse()
  
  return binario[:k], tf[:k], tfidf[:k], BM25[:k]

In [266]:
def indexes(model):
  return [doc for score,doc in model]

def intersection(a,b):
  return [elem for elem in a if elem in b]

def calcAP(query):
  relevant_docs = []

  for info in feedback[query]:
    row = dataframe.loc[dataframe.url == info['URL']]
    relevant_docs.append(row.index[0])
  
  binario, tf, tfidf, bm25 = create_models(query, 10)
  binario = indexes(binario)
  tf = indexes(tf)
  tfidf = indexes(tfidf)
  bm25 = indexes(bm25)
  
  binarioAP = len(intersection(binario, relevant_docs)) / len(binario)
  tfAP = len(intersection(tf, relevant_docs)) / len(tf)
  tf_idfAP = len(intersection(tfidf, relevant_docs)) / len(tfidf)
  BM25AP = len(intersection(bm25, relevant_docs)) / len(bm25)
  
  return binarioAP, tfAP, tf_idfAP, BM25AP
  
def calcMAP(queries):
  binarioSUM = 0
  TFSUM = 0
  tfidfSUM = 0
  bm25SUM = 0
  
  for consulta in queries:
    ap_binario, ap_tf, ap_tf_idf, ap_BM25 = calcAP(consulta)
    binarioSUM += ap_binario
    TFSUM += ap_tf
    tfidfSUM += ap_tf_idf
    bm25SUM += ap_BM25
  
  binarioMAP = round(binarioSUM / len(queries),2)
  tfMAP = round(TFSUM / len(queries),2)
  tf_idfMAP = round(tfidfSUM / len(queries),2)
  BM25MAP = round(bm25SUM / len(queries),2)
  
  return binarioMAP, tfMAP, tf_idfMAP, BM25MAP
list(feedback.keys())

['território palestino',
 'recessão mundial',
 'ditadura militar',
 'muro das lamentações',
 'brasil e argentina',
 'golpe militar',
 'governo bolsonaro',
 'ministro da economia',
 'prisão de Temer',
 'Congresso Nacional']

## Calculo do MAP para cada modelo

In [0]:
mapBinario, mapTF, mapTf_idf, mapBM25 = calcMAP(list(feedback.keys()))


In [268]:
mapBinario

0.08

In [269]:
mapTF

0.1

In [270]:
mapTf_idf

0.12

In [271]:
mapBM25

0.14

# 3. Repita Q2 usando a avaliação multi-nível DCG. Utilize o campo "level" do gabarito para o cálculo do DCG e do idealDCG. Use uma janela de 5 documentos

In [0]:
def dcg(model, docs):
  dcg = 0
  for i in range(1,len(model)+1):
    doc = model[i-1]
    lvl = docs[doc] if doc in docs else 0
    dcg += (2^lvl) / np.log2(i + 1)
    
  return dcg

def dcgModels(consulta):
  docs = {}

  for info in feedback[consulta]:
    row = dataframe.loc[dataframe.url == info['URL']]
    docs[row.index[0]] = info['level']
    
  binario, tf, tfidf, BM25 = create_models(consulta, 5)
  binario = indexes(binario)
  tf = indexes(tf)
  tfidf = indexes(tfidf)
  BM25 = indexes(BM25)
  
  binarioDCG = dcg(binario, docs)
  tfDCG = dcg(tf, docs)
  tf_idfDCG = dcg(tfidf, docs)
  BM25DCG = dcg(BM25, docs)
  
  return binarioDCG, tfDCG, tf_idfDCG, BM25DCG

In [0]:
binarioDCG, tfDCG, tf_idfDCG, BM25DCG = dcgModels('golpe militar')

In [274]:
binarioDCG

5.8969182377587845

In [275]:
tfDCG

8.05030102812575

In [276]:
tf_idfDCG

8.396918237758785

In [277]:
BM25DCG

8.396918237758785